# LLaMA Factory란?
- LLaMA Factory는 LLM 모델의 훈련을 손쉽게 파인튜닝 할 수 있습니다.
- `github`에 올려진 LLaMA Factory Repository를 클론한 내용을 토대로 마음대로 커스터마이징 하여 사용할 수 있습니다.

# LLaMA Factory Clone
라마 팩토리의 공식 깃헙은 https://github.com/hiyouga/LLaMA-Factory.git 입니다. 하지만 업데이트가 너무 잦아 코드 작동이 잘 안되는 경우가 있기 때문에, 미리 클론해 놓은 Github Repository를 사용합니다.

In [ ]:
!git clone https://github.com/llm-fine-tuning/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 12759, done.
remote: Counting objects: 100% (5570/5570), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 12759 (delta 5346), reused 5301 (delta 5301), pack-reused 7189 (from 1)
Receiving objects: 100% (12759/12759), 219.52 MiB | 12.46 MiB/s, done.
Resolving deltas: 100% (9431/9431), done.


In [ ]:
%cd LLaMA-Factory/

/content/LLaMA-Factory


In [ ]:
!ls

assets		    evaluation	    README_kr.md      src
CITATION.cff	    examples	    README.md	      tests
data		    LICENSE	    README_zh.md      train_sft.sh
docker-compose.yml  Makefile	    requirements.txt  tuning_example_rag.ipynb
Dockerfile	    merge.py	    scripts	      tuning_example_text_to_sql.ipynb
ds_z3_config.json   pyproject.toml  setup.py


# 중요 파일 설명⭐️
LLaMA Factory를 파인튜닝 하기 위해 반드시 설정해야 할 파일 및 디렉토리에 대해 설명합니다.

## data 디렉토리
`data` 디렉토리에는 다양한 데이터가 존재하며, 사용자는 이 `data` 디렉토리에 파인튜닝할 데이터를 `json` 형식으로 입력해야 합니다. 이 `json` 형식의 데이터는 추후 `template`과 결합되어 LLM 학습에 사용됩니다.

사용되는 `json` 데이터 형식은 `instruction`, `input`, `output` 세 가지 키가 반드시 포함되어야 합니다.

```json
{
    "instruction": "입력 텍스트: equipment_maintenance 테이블에서 모든 고유 장비 유형과 해당하는 총 유지 보수 빈도를 나열하세요.\n\nDDL statements:\nCREATE TABLE equipment_maintenance (equipment_type VARCHAR(255), maintenance_frequency INT);\n\n위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.",
    "input": "",
    "output": "쿼리 작성: SELECT equipment_type, SUM(maintenance_frequency) AS total_maintenance_frequency FROM equipment_maintenance GROUP BY equipment_type;"
},
{
    "instruction": "입력 텍스트: 남해양에서 발견되는 해양 생물 종은 몇 종인가요?\n\nDDL statements:\nCREATE TABLE marine_species (name VARCHAR(50), common_name VARCHAR(50), location VARCHAR(50));\n\n위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.",
    "input": "",
    "output": "쿼리 작성: SELECT COUNT(*) FROM marine_species WHERE location = 'Southern Ocean';"
}

```

- `instruction` : 유저가 챗봇에게 질문을 할 내용이 들어갑니다.
- `input` : `instruction`과 비슷하게 챗봇에게 전달할 내용이 들어가는데, 결과적으로 `instruction`과 `input`이 합쳐져서 챗봇에게 전달됩니다. 보통은 아무것도 없이 비워 놓아도 상관이 없습니다.
- `output` : 챗봇이 답변하는 내용입니다.

## dataset_info.json
새로운 데이터 파일(`json`)을 만들었다면, 이 데이터에 대한 정보도 추가해 줘야 합니다. `data` 디렉토리 내에 `dataset_info.json`에 추가된 새로운 `json` 파일에 대한 정보를 적어주면 됩니다.

```json
data/dataset_info.json

{
  "identity": {
    "file_name": "identity.json"
  },
  "text_to_sql_data": {
    "file_name": "text_to_sql_data.json"
  },
  "rag": {
    "file_name": "rag.json"
  },
  ...
}
```

## template.py
LLM의 챗봇 템플릿을 지정하는 파이썬 파일입니다. `src/llamafactory/data/template.py`에 있습니다.

템플릿은 각 LLM이 훈련 될 때 맞춰져 있기 때문에 특정 LLM을 학습시키기 위해서는 각 모델에 맞는 템플릿을 사용해야 학습이 원활해집니다.

아래 예시는 `default` 템플릿입니다. 아무 LLM이나 사용 가능합니다. (615 라인)

```python
_register_template(
    name="default",
    format_user=StringFormatter(slots=["Human: {{content}}\nAssistant: "]),
    format_system=StringFormatter(slots=["{{content}}\n"]),
    format_separator=EmptyFormatter(slots=["\n"]),
)
```

LLaMA-3 템플릿도 역시 존재합니다.(712 라인)
```python
_register_template(
    name="llama3",
    format_user=StringFormatter(
        slots=[
            (
                "<|start_header_id|>user<|end_header_id|>\n\n{{content}}<|eot_id|>"
                "<|start_header_id|>assistant<|end_header_id|>\n\n"
            )
        ]
    ),
    format_system=StringFormatter(
        slots=[{"bos_token"}, "<|start_header_id|>system<|end_header_id|>\n\n{{content}}<|eot_id|>"]
    ),
    format_observation=StringFormatter(
        slots=[
            (
                "<|start_header_id|>tool<|end_header_id|>\n\n{{content}}<|eot_id|>"
                "<|start_header_id|>assistant<|end_header_id|>\n\n"
            )
        ]
    ),
    default_system="You are a helpful assistant.",
    stop_words=["<|eot_id|>"],
    replace_eos=True,
)
```


`template.py`의 `_register_template` 함수를 이용해 사용자가 직접 커스터마이징 하는 것도 가능합니다.
```python
_register_template(
    name="llama3-rag",
    format_user=StringFormatter(
        slots=[
            (
                "<|start_header_id|>user<|end_header_id|>\n\n{{content}}<|eot_id|>"
                "<|start_header_id|>assistant<|end_header_id|>"
            )
        ]
    ),
    format_system=StringFormatter(
        slots=[{"bos_token"}, "<|start_header_id|>system<|end_header_id|>\n\n{{content}}<|eot_id|>"]
    ),
    default_system="당신은 검색된 문서를 바탕으로 질문에 답변하는 챗봇입니다.\n\n### 지시 사항 ###\n1. 검색 결과에 있는 내용을 바탕으로 질문에 답하세요.\n2. 만약 당신의 답변에서 검색 결과의 내용을 인용했다면, 답변 끝에 인용한 출처의 문서 id를 꼭 추가해 주세요. 문서 id는 검색 결과에 나와 있는 값을 사용하고, 이중 리스트 형태로 사용됩니다. 예시: [[doc1], [doc8]]\n3. 질문에 대한 답을 검색 결과에서 찾을 수 없다면 검색 결과에서는 해당 내용을 찾을 수 없다고 답변하고, 임의로 답변하려고 시도하지 마세요.",
    stop_words=["<|eot_id|>"],
    replace_eos=True,
)
```

결과적으로 `data` 디렉토리에 있는 `json`형태의 LLaMA Factory 입력 데이터의 형식과, 모델의 `ChatTemplate`이 합쳐져 LLM에서 요구하는 형식의 질문을 하게 됩니다.


```json
sample_data.json

{
    "instruction": "갈비찜이 뭐야??",
    "input": "",
    "output": "갈비찜은 한국 전통 음식으로, 부드럽게 익힌 갈비에 양념을 넣어 맛을 낸 요리입니다."
}
```

```python
default template

_register_template(
    name="default",
    format_user=StringFormatter(slots=["Human: {{content}}\nAssistant: "]),
    format_system=StringFormatter(slots=["{{content}}\n"]),
    format_separator=EmptyFormatter(slots=["\n"]),
)
```
```
합쳐진 템플릿

Human: 갈비찜이 뭐야??<|im_end|>
Assistant: 갈비찜은 한국 전통 음식으로, 부드럽게 익힌 갈비에 양념을 넣어 맛을 낸 요리입니다.<|im_end|>
```